<a href="https://colab.research.google.com/github/Kritika1303/Zeotap_assignment/blob/main/Kritika_Kritika_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***Importing Libraries***

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

***Merge the datasets***

In [25]:
# Load the datasets
customers = pd.read_csv('/content/drive/MyDrive/Zeotap_Assignment/Customers.csv')
products = pd.read_csv('/content/drive/MyDrive/Zeotap_Assignment/Products.csv')
transactions = pd.read_csv('/content/drive/MyDrive/Zeotap_Assignment/Transactions.csv')

# Merge the datasets: transactions -> customers -> products
merged_data = transactions.merge(customers, on='CustomerID', how='left')
merged_data = merged_data.merge(products, on='ProductID', how='left')

merged_data_df = pd.DataFrame(merged_data)
merged_data_df #show dataframe

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,459.86


In [26]:
merged_data_df.info() #for checking data type and null values if occuring

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price_x          1000 non-null   float64
 7   CustomerName     1000 non-null   object 
 8   Region           1000 non-null   object 
 9   SignupDate       1000 non-null   object 
 10  ProductName      1000 non-null   object 
 11  Category         1000 non-null   object 
 12  Price_y          1000 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB


In [27]:
merged_data_df.describe() #descriptive stats of numerical variables

,Quantity,TotalValue,Price_x,Price_y
count,1000.000000,1000.000000,1000.00000,1000.00000
mean,2.537000,689.995560,272.55407,272.55407
std,1.117981,493.144478,140.73639,140.73639
min,1.000000,16.080000,16.08000,16.08000
25%,2.000000,295.295000,147.95000,147.95000
50%,3.000000,588.880000,299.93000,299.93000
75%,4.000000,1011.660000,404.40000,404.40000
max,4.000000,1991.040000,497.76000,497.76000


***Feature Engineering***

In [29]:
# Total spending and purchase frequency per customer
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',         # Total spending
    'TransactionID': 'count',    # Purchase frequency
}).rename(columns={'TotalValue': 'TotalSpending', 'TransactionID': 'PurchaseFrequency'})

# Region encoding
region_encoding = pd.get_dummies(customers[['CustomerID', 'Region']], columns=['Region'])

# Merge features with customer features
customer_features = customer_features.merge(region_encoding, on='CustomerID', how='left')

# Product category preferences
category_pref = merged_data.groupby(['CustomerID', 'Category']).size().unstack(fill_value=0)
customer_features = customer_features.merge(category_pref, on='CustomerID', how='left')

# Normalize numeric features
scaler = MinMaxScaler()
numeric_columns = ['TotalSpending', 'PurchaseFrequency'] + list(category_pref.columns)
customer_features[numeric_columns] = scaler.fit_transform(customer_features[numeric_columns])

customer_features.head()

,CustomerID,TotalSpending,PurchaseFrequency,Region_Asia,Region_Europe,Region_North America,Region_South America,Books,Clothing,Electronics,Home Decor
0,C0001,0.308942,0.4,False,False,False,True,0.2,0.0,0.6,0.166667
1,C0002,0.168095,0.3,True,False,False,False,0.0,0.4,0.0,0.333333
2,C0003,0.249541,0.3,False,False,False,True,0.0,0.2,0.2,0.333333
3,C0004,0.497806,0.7,False,False,False,True,0.6,0.0,0.4,0.500000
4,C0005,0.184287,0.2,True,False,False,False,0.0,0.0,0.4,0.166667


***Similarity Calculations***

1. Cosine Similarity

In [30]:
# Compute cosine similarity between all customers
similarity_matrix = cosine_similarity(customer_features.iloc[:, 1:])  # Exclude 'CustomerID'

# Convert similarity matrix to a DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

similarity_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.148757,0.912828,0.913640,0.277709,0.857121,0.286186,0.444191,0.154506,0.130113,...,0.933645,0.961805,0.209539,0.399818,0.863093,0.225979,0.275987,0.136016,0.307497,0.292964
C0002,0.148757,1.000000,0.236468,0.246620,0.865951,0.203797,0.865104,0.353754,0.177034,0.245439,...,0.106458,0.146366,0.717364,0.261379,0.337430,0.249000,0.109563,0.100047,0.166190,0.938901
C0003,0.912828,0.236468,1.000000,0.861300,0.185528,0.912856,0.193747,0.395905,0.153959,0.174274,...,0.826080,0.928074,0.102043,0.308423,0.973213,0.251592,0.183878,0.107127,0.238150,0.274800
C0004,0.913640,0.246620,0.861300,1.000000,0.267072,0.899783,0.279104,0.556953,0.146108,0.208811,...,0.923114,0.855791,0.388824,0.525457,0.868746,0.375436,0.264657,0.114537,0.334811,0.388204
C0005,0.277709,0.865951,0.185528,0.267072,1.000000,0.117374,0.998994,0.311968,0.107007,0.064000,...,0.195331,0.194004,0.735864,0.255346,0.218016,0.146211,0.206280,0.097587,0.232730,0.886963


In [47]:
# Get the top 3 most similar customers for each customer
lookalike_results = {}

for customer in customer_features['CustomerID'][:20]:  # Only for the first 20 customers
    similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:4]  # Exclude the customer itself
    lookalike_results[customer] = list(similar_customers.index), list(similar_customers.values)

# Convert to DataFrame for export
lookalike_output = pd.DataFrame({
    'CustomerID': lookalike_results.keys(),
    'Lookalikes': [', '.join(cust_ids) for cust_ids, _ in lookalike_results.values()],
    'Scores': [', '.join([f'{score:.2f}' for score in scores]) for _, scores in lookalike_results.values()]
})
lookalike_output.to_csv('Lookalike.csv', index=False)
lookalike_output

,CustomerID,Lookalikes,Scores
0,C0001,"C0190, C0048, C0091","0.98, 0.98, 0.97"
1,C0002,"C0134, C0159, C0106","0.99, 0.98, 0.98"
2,C0003,"C0031, C0158, C0129","1.00, 1.00, 0.99"
3,C0004,"C0113, C0012, C0104","0.98, 0.98, 0.97"
4,C0005,"C0007, C0146, C0140","1.00, 0.98, 0.98"
5,C0006,"C0187, C0169, C0011","0.99, 0.97, 0.97"
6,C0007,"C0005, C0146, C0140","1.00, 0.98, 0.98"
7,C0008,"C0059, C0098, C0154","0.97, 0.96, 0.96"
8,C0009,"C0198, C0062, C0061","0.98, 0.97, 0.96"
9,C0010,"C0061, C0062, C0111","0.99, 0.96, 0.96"


2. Euclidean and Manhattan Similarity

In [48]:
# Cluster customers using KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(customer_features.iloc[:, 1:])  # Exclude 'CustomerID'
# Add cluster labels to the DataFrame
customer_features['Cluster'] = clusters
# Initialize dictionaries to store results for each distance metric
lookalike_results_euclidean = {}
lookalike_results_manhattan = {}
# Iterate over each cluster to find nearest neighbors within the cluster
for cluster_id in customer_features['Cluster'].unique():
    # Filter data for the current cluster
    cluster_data = customer_features[customer_features['Cluster'] == cluster_id].iloc[:, 1:-1]  # Exclude 'CustomerID' and 'Cluster'
    customer_ids = customer_features[customer_features['Cluster'] == cluster_id]['CustomerID']

    ## Euclidean Distance ##
    nbrs_euclidean = NearestNeighbors(n_neighbors=4, metric='euclidean').fit(cluster_data)
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(cluster_data)

    ## Manhattan Distance ##
    nbrs_manhattan = NearestNeighbors(n_neighbors=4, metric='manhattan').fit(cluster_data)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(cluster_data)

    for idx, cust_id in enumerate(customer_ids):
        # Store lookalike results for Euclidean distance
        similar_ids_euclidean = customer_ids.iloc[indices_euclidean[idx][1:]]  # Exclude itself
        scores_euclidean = 1 - distances_euclidean[idx][1:]  # Similarity (higher score means more similar)
        lookalike_results_euclidean[cust_id] = list(zip(similar_ids_euclidean, scores_euclidean))

        # Store lookalike results for Manhattan distance
        similar_ids_manhattan = customer_ids.iloc[indices_manhattan[idx][1:]]  # Exclude itself
        scores_manhattan = 1 - distances_manhattan[idx][1:]  # Similarity
        lookalike_results_manhattan[cust_id] = list(zip(similar_ids_manhattan, scores_manhattan))

# Convert results to DataFrame for easier visualization
# Convert lookalikes and scores to strings
def format_results(results_dict):
    formatted_results = []
    for cust_id, lookalikes_scores in results_dict.items():
        lookalikes = ', '.join([str(x[0]) for x in lookalikes_scores])
        scores = ', '.join([f"{x[1]:.2f}" for x in lookalikes_scores])
        formatted_results.append((cust_id, lookalikes, scores))
    return formatted_results

# Format the results for both Euclidean and Manhattan
formatted_euclidean = format_results(lookalike_results_euclidean)
formatted_manhattan = format_results(lookalike_results_manhattan)

# Create DataFrames from formatted results
lookalike_df_euclidean = pd.DataFrame(formatted_euclidean, columns=['CustomerID', 'Lookalikes (Euclidean)', 'Scores (Euclidean)'])
lookalike_df_manhattan = pd.DataFrame(formatted_manhattan, columns=['CustomerID', 'Lookalikes (Manhattan)', 'Scores (Manhattan)'])

# Merge the Euclidean and Manhattan results
lookalike_df = pd.merge(lookalike_df_euclidean, lookalike_df_manhattan, on='CustomerID')

# Display the first few rows of the results
lookalike_df.head()

,CustomerID,Lookalikes (Euclidean),Scores (Euclidean),Lookalikes (Manhattan),Scores (Manhattan)
0,C0001,"C0190, C0048, C0148","0.71, 0.71, 0.67","C0190, C0048, C0148","0.56, 0.55, 0.43"
1,C0003,"C0031, C0158, C0129","0.93, 0.91, 0.80","C0031, C0158, C0129","0.93, 0.91, 0.70"
2,C0004,"C0113, C0012, C0104","0.71, 0.66, 0.60","C0113, C0012, C0148","0.55, 0.32, 0.12"
3,C0006,"C0187, C0011, C0169","0.80, 0.65, 0.65","C0187, C0071, C0025","0.70, 0.43, 0.38"
4,C0009,"C0198, C0062, C0061","0.78, 0.73, 0.69","C0198, C0062, C0061","0.70, 0.56, 0.48"


On comparison **Cosine Similarity** gives better scores than Euclidean and Manhattan similarity